# Additional Tutorial 2: Work for multiple sections (STAGATE + Harmony)

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")

In [2]:
import STAGATE

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load Data

In [4]:
adata_list = {}

# Slide-seqV2

In [5]:
input_dir = '/home/zbdc/github/Data/Mouse_olfactory_bulb/Slide-seqV2'
counts_file = os.path.join(input_dir, 'Puck_200127_15.digital_expression.txt')
coor_file = os.path.join(input_dir, 'Puck_200127_15_bead_locations.csv')

In [6]:
counts = pd.read_csv(counts_file, sep='\t', index_col=0)
coor_df = pd.read_csv(coor_file, index_col=0)
print(counts.shape, coor_df.shape)

(21220, 21724) (21724, 3)


In [7]:
adata = sc.AnnData(counts.T)
adata.var_names_make_unique()
coor_df = coor_df.loc[adata.obs_names, ['xcoord', 'ycoord']]
adata.obsm["spatial"] = coor_df.to_numpy()

KeyError: "None of [Index(['TTTTTTTTTTTTTT', 'GCTAGGATTGTAAA', 'CACAACAACGTTGG', 'AATGACGGCAATGC',\n       'TATTTTAGATCTCA', 'CGCTAACGTCCTTA', 'TTCCCCGCTATCCT', 'TGATGGAAAAAGTC',\n       'ATCCCAAAATAATT', 'ATCGCTATGCTTTA',\n       ...\n       'TCCAGTCGACGGGG', 'AACCAGTCCCCTAA', 'GGGAATAAAACCGA', 'TAACATCATTCCTA',\n       'TCGACAGGTGGAGG', 'GCCGCCCGTTGGCT', 'ACAATAAGGGTCCT', 'TCTTCACTATCGCT',\n       'AAGAATTACTAAGC', 'AATCCACATCTTAT'],\n      dtype='object', length=21724)] are in the [index]"

In [ ]:
sc.pp.calculate_qc_metrics(adata, inplace=True)

In [ ]:
adata

In [ ]:
plt.rcParams["figure.figsize"] = (6,5)
sc.pl.embedding(adata, basis="spatial", color="log1p_total_counts",s=6, show=False)
plt.title('')
plt.axis('off')

In [ ]:
# can be downloaded from https://drive.google.com/drive/folders/10lhz5VY7YfvHrtV40MwaqLmWz56U9eBP?usp=sharing
used_barcode = pd.read_csv(os.path.join(input_dir, 'used_barcodes.txt'), sep='\t', header=None)
used_barcode = used_barcode[0]

In [ ]:
adata = adata[used_barcode,]
adata

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)
sc.pl.embedding(adata, basis="spatial", color="log1p_total_counts",s=10, show=False, title='Removing spots outside the main tissue area')

plt.axis('off')

In [ ]:
sc.pp.filter_genes(adata, min_cells=50)
print('After flitering: ', adata.shape)

In [ ]:
# make spot name unique
adata.obs_names = [x+'_SlideSeqV2' for x in adata.obs_names]

In [ ]:
adata_list['SlideSeqV2'] = adata.copy()

# Stereo-seq

In [ ]:
input_dir = '/home/zbdc/github/Data/Mouse olfactory bulb/Stereo-seq'
counts_file = os.path.join(input_dir, 'RNA_counts.tsv')
coor_file = os.path.join(input_dir, 'position.tsv')

In [ ]:
counts = pd.read_csv(counts_file, sep='\t', index_col=0)
coor_df = pd.read_csv(coor_file, sep='\t')
print(counts.shape, coor_df.shape)

In [ ]:
counts.columns = ['Spot_'+str(x) for x in counts.columns]
coor_df.index = coor_df['label'].map(lambda x: 'Spot_'+str(x))
coor_df = coor_df.loc[:, ['x','y']]

In [ ]:
coor_df.head()

In [ ]:
adata = sc.AnnData(counts.T)
adata.var_names_make_unique()

In [ ]:
adata

In [ ]:
coor_df = coor_df.loc[adata.obs_names, ['y', 'x']]
adata.obsm["spatial"] = coor_df.to_numpy()
sc.pp.calculate_qc_metrics(adata, inplace=True)

In [ ]:
plt.rcParams["figure.figsize"] = (5,4)
sc.pl.embedding(adata, basis="spatial", color="n_genes_by_counts", show=False)
plt.title("")
plt.axis('off')

In [ ]:
used_barcode = pd.read_csv(os.path.join(input_dir, 'used_barcodes.txt'), sep='\t', header=None)
used_barcode = used_barcode[0]
adata = adata[used_barcode,]

In [ ]:
adata

In [ ]:
plt.rcParams["figure.figsize"] = (5,4)
sc.pl.embedding(adata, basis="spatial", color="n_genes_by_counts", show=False)
plt.title("")
plt.axis('off')

In [ ]:
sc.pp.filter_genes(adata, min_cells=50)
print('After flitering: ', adata.shape)

In [ ]:
# make spot name unique
adata.obs_names = [x+'_StereoSeq' for x in adata.obs_names]

In [ ]:
adata_list['StereoSeq'] = adata.copy()

# Constructing the spatial network for each secion

## Slide-seqV2

In [ ]:
STAGATE.Cal_Spatial_Net(adata_list['SlideSeqV2'], rad_cutoff=50)
STAGATE.Stats_Spatial_Net(adata_list['SlideSeqV2'])

In [ ]:
STAGATE.Cal_Spatial_Net(adata_list['StereoSeq'], rad_cutoff=50)
STAGATE.Stats_Spatial_Net(adata_list['StereoSeq'])

# Note that the spatial network are saved in adata.uns[‘Spatial_Net’], which can be conbat directly for multiple sections.

In [ ]:
adata_list['SlideSeqV2'].uns['Spatial_Net']

# Conbat the scanpy objects and spatial networks

In [ ]:
adata = sc.concat([adata_list['SlideSeqV2'], adata_list['StereoSeq']], keys=None)

In [ ]:
adata.uns['Spatial_Net'] = pd.concat([adata_list['SlideSeqV2'].uns['Spatial_Net'], adata_list['StereoSeq'].uns['Spatial_Net']])

In [ ]:
STAGATE.Stats_Spatial_Net(adata)

# Normalization

In [ ]:
#Normalization
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Running STAGATE

In [ ]:
adata = STAGATE.train_STAGATE(adata, alpha=0)

In [ ]:
sc.pp.neighbors(adata, use_rep='STAGATE')
sc.tl.umap(adata)

In [ ]:
adata.obs['Tech'] = [x.split('_')[-1] for x in adata.obs_names]

In [ ]:
plt.rcParams["figure.figsize"] = (3, 3)
sc.pl.umap(adata, color='Tech', title='Unintegrated')

# Run Harmony on the STAGATE represention

In [ ]:
import harmonypy as hm 

In [ ]:
data_mat = adata.obsm['STAGATE'].copy()
meta_data = adata.obs.copy()

In [ ]:
# Run Harmony
ho = hm.run_harmony(data_mat, meta_data, ['Tech'])

In [ ]:
# Write the adjusted PCs to a new file.
res = pd.DataFrame(ho.Z_corr)
res.columns = adata.obs_names

In [ ]:
adata_Harmony = sc.AnnData(res.T)

In [ ]:
adata_Harmony.obsm['spatial'] = pd.DataFrame(adata.obsm['spatial'], index=adata.obs_names).loc[adata_Harmony.obs_names,].values
adata_Harmony.obs['Tech'] = adata.obs.loc[adata_Harmony.obs_names, 'Tech']

In [ ]:
sc.pp.neighbors(adata_Harmony)
sc.tl.umap(adata_Harmony)

In [ ]:
sc.tl.louvain(adata_Harmony, resolution=0.8)

In [ ]:
plt.rcParams["figure.figsize"] = (3, 3)
sc.pl.umap(adata_Harmony, color='Tech', title='STAGATE + Harmony')

In [ ]:
plt.rcParams["figure.figsize"] = (3, 3)
sc.pl.umap(adata_Harmony, color='louvain')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
it=0
for temp_tech in ['StereoSeq', 'SlideSeqV2']:
    temp_adata = adata_Harmony[adata_Harmony.obs['Tech']==temp_tech, ]
    if it == 1:
        sc.pl.embedding(temp_adata, basis="spatial", color="louvain",s=6, ax=axs[it],
                        show=False, title=temp_tech)
    else:
        sc.pl.embedding(temp_adata, basis="spatial", color="louvain",s=6, ax=axs[it], legend_loc=None,
                        show=False, title=temp_tech)
    it+=1